In [ ]:
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [5]:
import mlflow
from mlflow.models import infer_signature

In [16]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/examples/sklearn_elasticnet_wine/wine-quality.csv")
data["quality"].unique()

array([6, 5, 7, 8, 4, 3, 9], dtype=int64)

In [20]:
X = data.drop('quality', axis=1)
y = data['quality']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
signature = infer_signature(X_train,y_train)


c:\Users\Paarth\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense

def build_ann_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

In [ ]:
model=build_ann_model(X_train_scaled.shape[1])
model.compile(optimizer='adam', 
                loss='mean_squared_error',
                metrics=['mean_squared_error'])
with mlflow.start_run:
    model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_data=(X_test_scaled, y_test),verbose=0)
    mlflow.keras.log_model(model, "model", signature=signature)
    mlflow.log_metric("mse", model.evaluate(X_test_scaled, y_test)[1])
    mlflow.log_param("epochs", 20)
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("layers", 2)
    mlflow.log_param("units", 64)
    mlflow.log_param("activation", "relu")
    mlflow.log_param("loss", "mean_squared_error")
    mlflow.log_param("optimizer", "adam")
    mlflow.log_param("metrics", "mean_squared_error")
    mlflow.log_param("input_dim", X_train_scaled.shape[1])
    mlflow.log_param("output_dim", 1)

    mlflow.keras.log_model(model, "model", signature=signature)
    return {"loss": model.evaluate(X_test_scaled, y_test)[1], "status": STATUS_OK}

Epoch 1/20


c:\Users\Paarth\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 41.5198 - mean_squared_error: 41.5198 - val_loss: 2.6356 - val_mean_squared_error: 2.6356
Epoch 2/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5191 - mean_squared_error: 1.5191 - val_loss: 0.7857 - val_mean_squared_error: 0.7857
Epoch 3/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7569 - mean_squared_error: 0.7569 - val_loss: 0.6958 - val_mean_squared_error: 0.6958
Epoch 4/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6352 - mean_squared_error: 0.6352 - val_loss: 0.6657 - val_mean_squared_error: 0.6657
Epoch 5/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6753 - mean_squared_error: 0.6753 - val_loss: 0.6537 - val_mean_squared_error: 0.6537
Epoch 6/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7006 - mean_squared_error: 0.7006 - val_loss: 0.6423 - val_mean_squared_error: 0.6423
Epoch 7/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6359 - mean_squared_error: 0.6359 - val_loss: 0.8040 - val_

SyntaxError: 'return' outside function (2654783836.py, line 23)